In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
set2 = pd.read_csv('lv2-2402.csv')

In [3]:
set2.shape

(7488, 7)

In [4]:
set2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7488 entries, 0 to 7487
Data columns (total 7 columns):
PID             7488 non-null int64
USER_ID         7488 non-null int64
POST_ID         7488 non-null int64
CREATED_DATE    7488 non-null object
LIKES           7488 non-null object
NUM_HASHTAG     7488 non-null int64
LIKE_TYPE       4730 non-null object
dtypes: int64(4), object(3)
memory usage: 409.6+ KB


In [5]:
set2

,PID,USER_ID,POST_ID,CREATED_DATE,LIKES,NUM_HASHTAG,LIKE_TYPE
0,1,2,1,2023-04-13,yes,1,heart emoji
1,2,3,1,2023-04-13,no,2,NaN
2,3,5,1,2023-04-13,no,4,fire
3,4,14,1,2023-04-13,yes,2,laughing
4,5,17,1,2023-04-13,yes,1,NaN
...,...,...,...,...,...,...,...
7483,7484,82,257,2023-04-21,no,5,fire
7484,7485,91,257,2023-04-21,no,6,clap
7485,7486,93,257,2023-04-21,yes,2,NaN
7486,7487,95,257,2023-04-21,yes,1,thumbs up


### Q1

In [6]:
df4 = set2.copy()
#1-1.
agg1 = df4.groupby(['POST_ID','CREATED_DATE','LIKES'])['USER_ID'].count().reset_index()
agg1 = agg1[agg1['LIKES']=='yes']
agg1 = agg1[['POST_ID','CREATED_DATE','USER_ID']]
agg1.columns = ['POST_ID','CREATED_DATE','NUM_LIKES']
agg2 = df4.groupby(['POST_ID','CREATED_DATE'])['USER_ID'].count().reset_index()
agg2.columns = ['POST_ID','CREATED_DATE','NUM_USER']
agg24 = pd.merge(agg1, agg2, how='left',on=['POST_ID','CREATED_DATE'])
print(agg24.shape)

(257, 4)


In [7]:
agg24

,POST_ID,CREATED_DATE,NUM_LIKES,NUM_USER
0,1,2023-04-13,13,25
1,2,2023-04-13,15,31
2,3,2023-04-13,14,27
3,4,2023-04-13,16,32
4,5,2023-04-13,13,27
...,...,...,...,...
252,253,2023-04-21,13,26
253,254,2023-04-21,13,26
254,255,2023-04-21,15,29
255,256,2023-04-21,14,29


In [8]:
# 1-2.
yes = df4[df4['LIKES']=='yes']
map_yes = {'heart emoji':5, 'thumbs up':4, 'clap':3, 'laughing':2, 'fire':0}
yes['LIKE_TYPE']=yes['LIKE_TYPE'].map(map_yes)
yes.fillna(1, inplace=True)

In [9]:
no = df4[df4['LIKES']=='no']
map_no = {'fire':-5, 'heart emoji':0, 'thumbs up':0, 'clap':0, 'laughing':0}
no['LIKE_TYPE']=no['LIKE_TYPE'].map(map_no)
no.fillna(-1, inplace=True)

In [10]:
score = pd.concat([yes,no], axis=0).sort_values('PID')
score2 = score.groupby(['POST_ID'])['LIKE_TYPE'].sum().reset_index()
agg24_1=pd.merge(agg24, score2, on='POST_ID', how='left')
agg24_1.columns=['POST_ID', 'CREATED_DATE', 'NUM_LIKES', 'NUM_USER', 'LIKE_SCORE']

In [11]:
score

,PID,USER_ID,POST_ID,CREATED_DATE,LIKES,NUM_HASHTAG,LIKE_TYPE
0,1,2,1,2023-04-13,yes,1,5.0
1,2,3,1,2023-04-13,no,2,-1.0
2,3,5,1,2023-04-13,no,4,-5.0
3,4,14,1,2023-04-13,yes,2,2.0
4,5,17,1,2023-04-13,yes,1,1.0
...,...,...,...,...,...,...,...
7483,7484,82,257,2023-04-21,no,5,-5.0
7484,7485,91,257,2023-04-21,no,6,0.0
7485,7486,93,257,2023-04-21,yes,2,1.0
7486,7487,95,257,2023-04-21,yes,1,4.0


In [12]:
agg24_1

,POST_ID,CREATED_DATE,NUM_LIKES,NUM_USER,LIKE_SCORE
0,1,2023-04-13,13,25,0.0
1,2,2023-04-13,15,31,5.0
2,3,2023-04-13,14,27,23.0
3,4,2023-04-13,16,32,22.0
4,5,2023-04-13,13,27,18.0
...,...,...,...,...,...
252,253,2023-04-21,13,26,31.0
253,254,2023-04-21,13,26,26.0
254,255,2023-04-21,15,29,32.0
255,256,2023-04-21,14,29,6.0


In [13]:
# 1-3.
agg24_1['POPULAR']=agg24_1['NUM_LIKES']+(agg24_1['LIKE_SCORE']/agg24_1['NUM_USER'])
ans24 = agg24_1.sort_values('POPULAR', ascending=False).iloc[0].POPULAR

print(ans24)
print('Q1:',round(ans24,2))

20.82051282051282
Q1: 20.82


### Q2

In [14]:
df5 = set2.copy()
# 2-1.
df51 = df5.groupby(['USER_ID'])['POST_ID'].count().reset_index()
df51.columns = ['USER_ID','NUM_POST']
df52 = df5.groupby(['USER_ID','LIKES'])['POST_ID'].count().reset_index()
df52=df52[df52['LIKES']=='yes']
df52=df52[['USER_ID','POST_ID']]
df52.columns=['USER_ID','NUM_LIKE']
agg25 = pd.merge(df51,df52, how='left',on='USER_ID')
agg25

,USER_ID,NUM_POST,NUM_LIKE
0,2,66,29
1,3,67,25
2,4,64,32
3,5,257,135
4,6,62,31
...,...,...,...
72,96,75,42
73,97,68,41
74,98,65,28
75,99,68,39


In [15]:
# 2-2.
from sklearn.preprocessing import StandardScaler
scale=pd.DataFrame(StandardScaler().fit(agg25[['NUM_POST','NUM_LIKE']]).transform(agg25[['NUM_POST','NUM_LIKE']]), columns=['NUM_POST_S','NUM_LIKE_S'])
agg25s = pd.concat([agg25['USER_ID'],scale], axis=1)

# 2-3.
from sklearn.cluster import KMeans

cluster = KMeans(n_clusters=2, random_state = 1234)

agg25s['cls'] = cluster.fit_predict(agg25s[['NUM_POST_S','NUM_LIKE_S']])
agg25s

df53 = df5.merge(agg25s,on=['USER_ID'])

tmp = df53.groupby(by='cls').mean()['NUM_HASHTAG']

ans25 = np.abs(tmp[1]-tmp[0])

print(ans25)
print('Q2: ', round(ans25,3))

0.025986625745112235
Q2:  0.026


### Q3

In [16]:
df6f = set2.copy()

In [18]:
# 3-1.
agg1=df6f.groupby(['POST_ID','CREATED_DATE'])['USER_ID'].count().reset_index()
agg1.columns = ['POST_ID','CREATED_DATE','NUM_USER']
agg1 = agg1.sort_values('POST_ID')
agg2=df6f.groupby(['POST_ID','CREATED_DATE','LIKES'])['USER_ID'].count().reset_index()
agg2 = agg2[agg2['LIKES']=='yes'].reset_index()
agg2 = agg2[['POST_ID','CREATED_DATE','USER_ID']]
agg2.columns = ['POST_ID','CREATED_DATE','NUM_LIKES']
agg2 = agg2.sort_values('POST_ID')
agg3 = df6f.groupby(['POST_ID','CREATED_DATE'])['NUM_HASHTAG'].mean().reset_index()
agg3.columns = ['POST_ID','CREATED_DATE','AVG_HASHTAG']
agg3 = agg3.sort_values('POST_ID')

agg26 = pd.concat([agg1,agg2['NUM_LIKES'],agg3['AVG_HASHTAG']], axis=1)
agg26

,POST_ID,CREATED_DATE,NUM_USER,NUM_LIKES,AVG_HASHTAG
0,1,2023-04-13,25,13,2.360000
1,2,2023-04-13,31,15,2.387097
2,3,2023-04-13,27,14,2.481481
3,4,2023-04-13,32,16,2.281250
4,5,2023-04-13,27,13,2.629630
...,...,...,...,...,...
252,253,2023-04-21,26,13,2.384615
253,254,2023-04-21,26,13,2.346154
254,255,2023-04-21,29,15,2.310345
255,256,2023-04-21,29,14,2.586207


In [19]:
# 3-2.
agg26['POPULAR']=agg26['NUM_USER'].apply(lambda x : 1 if x>=31 else 0)

In [20]:
# 3-3.
from datetime import datetime

agg26['WEEKDAY']=agg26['CREATED_DATE'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d').strftime('%w'))
agg26['WEEKDAY']=agg26['WEEKDAY'].astype('int')

In [21]:
# 3-4
from sklearn.naive_bayes import GaussianNB

xtrain = agg26[['NUM_LIKES','AVG_HASHTAG', 'WEEKDAY']]
ytrain = agg26['POPULAR']

naive = GaussianNB().fit(xtrain, ytrain)

In [22]:
# 3-5

xtest = pd.DataFrame([[16, 2.42, 5]], columns=xtrain.columns)
pred = naive.predict_proba(xtest)

In [23]:
ans26 = pred[0][1]
print('Q3 : ', ans26)

Q3 :  0.7761246747414914


# Answer Summary

In [24]:
print('Q1:', ans24)
print('Q2:', ans25)
print('Q3:', ans26)

Q1: 20.82051282051282
Q2: 0.025986625745112235
Q3: 0.7761246747414914


In [25]:
print('Q1:', round(ans24, 2))
print('Q2:', round(ans25, 4))
print('Q3:', round(ans26, 3))

Q1: 20.82
Q2: 0.026
Q3: 0.776
